In [ ]:
# Setting up cuda in colab
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

# Caution!!

A warp is a set of threads that perform instructions in lockstep. If not programmed correctly would lead to thread divergence.

In [ ]:
%%writefile cpvec.cu
#include <iostream>
#include <cuda.h>

#define N 8192

using namespace std;

__global__ void copy_vector(float *x, float *y){
        int id = blockIdx.x*blockDim.x + threadIdx.x;
        y[id] = x[id];
}

int main(){
    float X_hs[N], *X_dev;
    float Y_hs[N], *Y_dev;

    // Initialise vector
    for(int i=0;i<N;i++){
        X_hs[i] = i * 1.0;
        Y_hs[i] = 0.0;
    }
    // Allocate memory in GPU (device)
    cudaMalloc(&X_dev, N * sizeof(float));
    cudaMalloc(&Y_dev, N * sizeof(float));

    // Transfer data to GPU
    cudaMemcpy(X_dev, X_hs, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(Y_dev, Y_hs, N * sizeof(float), cudaMemcpyHostToDevice);

    // Launch Kernel

     copy_vector<<<32,256>>>(X_dev, Y_dev);

    // Transfer data back to CPU
    cudaMemcpy(Y_hs, Y_dev, N * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(X_hs);
    cudaFree(Y_hs);

    return 0;
}

In [ ]:
!nvcc -o cpvec cpvec.cu

In [ ]:
!nvprof ./cpvec

In [ ]:
%%writefile man-vec.cu
#include <iostream>
#include <cuda.h>

#define N 65536

using namespace std;

__global__ void manipulate_vector_v1(float *x, float *y){
        int id = blockIdx.x*blockDim.x + threadIdx.x;
        if (id%4 == 0)
          y[id] = x[id];
        else{
          if (id%4 == 1)
            y[id] = x[id]+1;
          else{
            if (id%4 == 2)
              y[id] = x[id]+2;
            else
              y[id] = x[id]+3;
          }
        }
}

__global__ void manipulate_vector_v2(float *x, float *y){
        int id = blockIdx.x*blockDim.x + threadIdx.x;
        y[id] = x[id] + (id%4);
}

int main(){
    float X_hs[N], *X_dev;
    float Y_hs[N], *Y_dev;

    // Initialise vector
    for(int i=0;i<N;i++){
        X_hs[i] = i * 1.0;
        Y_hs[i] = 0.0;
    }
    // Allocate memory in GPU (device)
    cudaMalloc(&X_dev, N * sizeof(float));
    cudaMalloc(&Y_dev, N * sizeof(float));

    // Transfer data to GPU
    cudaMemcpy(X_dev, X_hs, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(Y_dev, Y_hs, N * sizeof(float), cudaMemcpyHostToDevice);

    // Launch Kernel

     manipulate_vector_v1<<<256,256>>>(X_dev, Y_dev);
     manipulate_vector_v2<<<256,256>>>(X_dev, Y_dev);

    // Transfer data back to CPU
    cudaMemcpy(Y_hs, Y_dev, N * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(X_hs);
    cudaFree(Y_hs);

    return 0;
}

In [ ]:
!nvcc -o mvec man-vec.cu
!nvprof ./mvec


## Memory

In [ ]:
%%writefile trans-mat.cu
#include <iostream>
#include <cuda.h>

#define N 65536
const int TILE_DIM = 32;
const int BLOCK_ROWS = 8;

using namespace std;

__global__ void transpose_v1(float *odata, const float *idata)
{
  int x = blockIdx.x * TILE_DIM + threadIdx.x;
  int y = blockIdx.y * TILE_DIM + threadIdx.y;
  int width = gridDim.x * TILE_DIM;

  for (int j = 0; j < TILE_DIM; j+= BLOCK_ROWS)
    odata[x*width + (y+j)] = idata[(y+j)*width + x];
}

__global__ void transpose_v2(float *odata, const float *idata)
{
  __shared__ float tile[TILE_DIM][TILE_DIM];

  int x = blockIdx.x * TILE_DIM + threadIdx.x;
  int y = blockIdx.y * TILE_DIM + threadIdx.y;
  int width = gridDim.x * TILE_DIM;

  for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS)
     tile[threadIdx.y+j][threadIdx.x] = idata[(y+j)*width + x];

  __syncthreads();

  x = blockIdx.y * TILE_DIM + threadIdx.x;  // transpose block offset
  y = blockIdx.x * TILE_DIM + threadIdx.y;

  for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS)
     odata[(y+j)*width + x] = tile[threadIdx.x][threadIdx.y + j];
}

int main(){
  const int nx = 1024;
  const int ny = 1024;
  const int mem_size = nx*ny*sizeof(float);

  dim3 dimGrid(nx/TILE_DIM, ny/TILE_DIM, 1);
  dim3 dimBlock(TILE_DIM, BLOCK_ROWS, 1);

  float *h_idata = (float*)malloc(mem_size);
  float *h_cdata = (float*)malloc(mem_size);
  float *h_tdata = (float*)malloc(mem_size);

  float *d_idata, *d_cdata, *d_tdata;
  cudaMalloc(&d_idata, mem_size);
  cudaMalloc(&d_cdata, mem_size);
  cudaMalloc(&d_tdata, mem_size);



  // Sample matrix
  for (int j = 0; j < ny; j++)
    for (int i = 0; i < nx; i++)
      h_idata[j*nx + i] = j*nx + i;

  cudaMemcpy(d_idata, h_idata, mem_size, cudaMemcpyHostToDevice);
  transpose_v1<<<dimGrid, dimBlock>>>(d_tdata, d_idata);
  cudaMemcpy(h_tdata, d_tdata, mem_size, cudaMemcpyDeviceToHost);

  cudaMemcpy(d_idata, h_idata, mem_size, cudaMemcpyHostToDevice);
  transpose_v2<<<dimGrid, dimBlock>>>(d_tdata, d_idata);
  cudaMemcpy(h_tdata, d_tdata, mem_size, cudaMemcpyDeviceToHost);

    return 0;
}

In [ ]:
!nvcc -o tmat trans-mat.cu
!nvprof ./tmat